In [1]:
from sys import path
path.append("./deep-belief-network-master/")
from dbn import SupervisedDBNRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import os

In [2]:
def RMSE(data1,data2):
    return np.sqrt(np.mean((data1-data2)*(data1-data2)))
def MAE(data1,data2):
    return np.mean(np.abs(data1-data2))
def BIAS(data1,data2):
    return np.mean(data1-data2)
def UBRMSE(data1,data2):
    rmse=RMSE(data1,data2)
    bias=BIAS(data1,data2)
    return np.sqrt(rmse*rmse-bias*bias)
def PCC(data1,data2):
    return pearsonr(data1,data2)[0]

In [22]:
def DBNTrain(datapath,hidden,rbm_lr,lr,rbm_epoch,epoch,batchsize):
    data=pd.read_csv(datapath)
    print(data.shape)
    predictors=data[["Ele","Albedo","SSM","UW","LST","NDVI","DOY","AT"]].values
    outputvar=data["Mean"].values
    # Splitting data
    X_train, X_test, Y_train, Y_test = train_test_split(predictors, outputvar, test_size=0.2,random_state=888)
    # Data scaling
    min_max_scaler = MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    # Training                                               
    regressor = SupervisedDBNRegression(hidden_layers_structure=hidden,
                                        learning_rate_rbm=rbm_lr,
                                        learning_rate=lr,
                                        n_epochs_rbm=rbm_epoch,
                                        n_iter_backprop=epoch,
                                        batch_size=batchsize,
                                        activation_function='relu')
    regressor.fit(X_train, Y_train)
    X_test = min_max_scaler.transform(X_test)
    Y_pred = regressor.predict(X_test)
    print(PCC(Y_pred[:,0],Y_test))
    return regressor,min_max_scaler

In [23]:
# Example of Moodel Training
datapath="./Data/TrainingSet.csv"
hidden=[20,15,10]
rbm_lr=0.005
lr=0.001
rbm_epoch=50
epoch=300
batchsize=16
model,scaler=DBNTrain(datapath,hidden,rbm_lr,lr,rbm_epoch,epoch,batchsize)

(63427, 11)
[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.165896
>> Epoch 2 finished 	RBM Reconstruction error 0.115320
>> Epoch 3 finished 	RBM Reconstruction error 0.104636
>> Epoch 4 finished 	RBM Reconstruction error 0.101223
>> Epoch 5 finished 	RBM Reconstruction error 0.098523
>> Epoch 6 finished 	RBM Reconstruction error 0.093818
>> Epoch 7 finished 	RBM Reconstruction error 0.092913
>> Epoch 8 finished 	RBM Reconstruction error 0.090705
>> Epoch 9 finished 	RBM Reconstruction error 0.088216
>> Epoch 10 finished 	RBM Reconstruction error 0.085728
>> Epoch 11 finished 	RBM Reconstruction error 0.084043
>> Epoch 12 finished 	RBM Reconstruction error 0.080238
>> Epoch 13 finished 	RBM Reconstruction error 0.079068
>> Epoch 14 finished 	RBM Reconstruction error 0.076152
>> Epoch 15 finished 	RBM Reconstruction error 0.075495
>> Epoch 16 finished 	RBM Reconstruction error 0.073380
>> Epoch 17 finished 	RBM Reconstruction error 0.073565
>> Epoch 18 finish

>> Epoch 48 finished 	RBM Reconstruction error 0.022217
>> Epoch 49 finished 	RBM Reconstruction error 0.022211
>> Epoch 50 finished 	RBM Reconstruction error 0.021493
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 1 finished 	ANN training loss 16.339051
>> Epoch 2 finished 	ANN training loss 4.646033
>> Epoch 3 finished 	ANN training loss 4.317397
>> Epoch 4 finished 	ANN training loss 4.198774
>> Epoch 5 finished 	ANN training loss 4.096930
>> Epoch 6 finished 	ANN training loss 4.019317
>> Epoch 7 finished 	ANN training loss 3.984079
>> Epoch 8 finished 	ANN training loss 3.931310
>> Epoch 9 finished 	ANN training loss 3.901430
>> Epoch 10 finished 	ANN training loss 3.859963
>> Epoch 11 finished 	ANN training loss 3.843294
>> Epoch 12 finished 	ANN training loss 3.803885
>> Epoch 13 finished 	ANN training loss 3.785922
>> Epoch 14 finished 	ANN training loss 3.758370
>> Epoch 15 finished 	ANN training loss 3.738430
>> Epoch 16 finished 	ANN training loss 3.694336
>> Epo

>> Epoch 163 finished 	ANN training loss 2.943902
>> Epoch 164 finished 	ANN training loss 2.937106
>> Epoch 165 finished 	ANN training loss 2.937919
>> Epoch 166 finished 	ANN training loss 2.947441
>> Epoch 167 finished 	ANN training loss 2.943667
>> Epoch 168 finished 	ANN training loss 2.942238
>> Epoch 169 finished 	ANN training loss 2.936113
>> Epoch 170 finished 	ANN training loss 2.939137
>> Epoch 171 finished 	ANN training loss 2.931809
>> Epoch 172 finished 	ANN training loss 2.934334
>> Epoch 173 finished 	ANN training loss 2.932894
>> Epoch 174 finished 	ANN training loss 2.933144
>> Epoch 175 finished 	ANN training loss 2.933511
>> Epoch 176 finished 	ANN training loss 2.931760
>> Epoch 177 finished 	ANN training loss 2.932592
>> Epoch 178 finished 	ANN training loss 2.928277
>> Epoch 179 finished 	ANN training loss 2.929115
>> Epoch 180 finished 	ANN training loss 2.919946
>> Epoch 181 finished 	ANN training loss 2.925860
>> Epoch 182 finished 	ANN training loss 2.926546


In [24]:
# Validation of the model performance
stds=os.listdir("./Data/ValidationStation/")
pccs=[]
rmses=[]
maes=[]
bias=[]
res,res1,res2=[],[],[]
stations=[]
for std in stds:
    data=pd.read_csv("./Data/ValidationStation/"+std)
    invar=data[["Ele","Albedo","SSM","UW","LST","NDVI","DOY","AT"]].values
    outvar=data["Mean"].values
    X = scaler.transform(invar)
    Y_p = model.predict(X)
    pccs.append(pearsonr(Y_p[:,0],outvar)[0])
    rmses.append(np.sqrt(mean_squared_error(outvar, Y_p[:,0])))
    maes.append(np.mean(np.abs(Y_p[:,0]-outvar)))
    bias.append(np.mean(Y_p[:,0]-outvar))
print(np.mean(np.array(pccs)))
print(np.mean(np.array(rmses)))
print(np.mean(np.array(maes)))

0.978197337437166
1.9898964643063999
1.6168324667176026
